In [1]:
import pandas as pd
from database.market import Market
import matplotlib.pyplot as plt
from processor.processor import Processor as p
from datetime import datetime
from statistics import variance
from tqdm import tqdm

In [2]:
market = Market()

In [3]:
market.connect()
prices = market.retrieve("prices")
pdr_prices = market.retrieve("pdr_prices")
prices = p.column_date_processing(prices)
pdr_prices = p.column_date_processing(pdr_prices)
market.disconnect()

E:\projects\longshot_venv\longshot\processor\processor.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["week"] = data["date"].dt.week
E:\projects\longshot_venv\longshot\processor\processor.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["week"] = data["date"].dt.week


In [4]:
prices = prices.groupby(["date","ticker"]).mean().reset_index()
pdr_prices = pdr_prices.groupby(["date","ticker"]).mean().reset_index()

In [5]:
start_date = datetime(2012,1,1)
end_date = datetime.now()

In [6]:
analysis = []
for ticker in tqdm(prices["ticker"].unique()):
    ticker_data = prices[(prices["ticker"]==ticker) & (prices["date"]>=start_date)].sort_values("date")
    pdr_data = pdr_prices[(pdr_prices["ticker"]==ticker) & (pdr_prices["date"]>=start_date)].sort_values("date")
    merged = ticker_data.merge(pdr_data,on="date")
    merged["diff"] = abs(merged["adjclose_x"] - merged["adjclose_y"]) /  merged["adjclose_y"]
#     var = sum((ticker_data["adjclose"] - pdr_data["adjclose"]) / pdr_data["adjclose"]) / pdr_data["adjclose"].index.size
    analysis.append({"ticker":ticker
                     ,"tiingo_count":ticker_data.index.size
                     ,"pdr_count":pdr_data.index.size
                     ,"count_diff":abs(ticker_data.index.size-pdr_data.index.size)
                     ,"var":merged["diff"].mean()})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 497/497 [01:09<00:00,  7.16it/s]


In [7]:
a = pd.DataFrame(analysis)

In [8]:
deactivated_tickers = a[(a["var"]>=0.005) | (a["count_diff"]>=10) | (a["pdr_count"]< 2670) | (a["tiingo_count"]< 2670)]

In [9]:
deactivated_tickers

,ticker,tiingo_count,pdr_count,count_diff,var
36,APD,2690,2690,0,0.036205
38,APTV,2690,2690,0,0.009397
43,AVGO,2690,2690,0,0.008503
68,CAG,2690,2690,0,0.007747
71,CB,2690,2690,0,0.015322
...,...,...,...,...,...
492,CARR,625,625,0,0.000025
493,OTIS,625,625,0,0.000061
494,VTRS,457,2690,2233,0.000102
495,OGN,323,334,11,0.000220


In [10]:
market.connect()
market.store("deactivated_tickers",deactivated_tickers)
market.disconnect()